<h1>Player Prediction Machine Learning Models - Batting</h1>

<h4>Import Dependencies</h4>

In [27]:
%matplotlib inline

In [28]:
import os
import csv
import pandas as pd

import sqlite3
import csv
from sqlalchemy import create_engine

import matplotlib.pyplot as plt
import numpy as np

from config import pgPassword

<h4>Create a connection to SQL database</h4>

In [29]:
pg_user = 'postgres'
pg_password = pgPassword
db_name = 'baseball_db'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

<h4>Read in databases tables as DatFrames</h4>

In [30]:
teamsStatsDF = pd.read_sql_table("Team-Stats", con = engine)
battingDF = pd.read_sql_table("Batting", con = engine)
pitchingDF = pd.read_sql_table("Pitching", con = engine)
playersDF = pd.read_sql_table("Players", con = engine)
franchisesDF = pd.read_sql_table("Franchises", con = engine)
salariesDF = pd.read_sql_table("Salaries", con = engine)
teamsDF = pd.read_sql_table("Teams", con = engine)
fPlayersDF = pd.read_sql_table("FranchisePlayers", con = engine)

In [31]:
battingDF.head()

,yearID,stint,G,R,H,HR,BB,IBB,SO,SB,fpID
0,1954,1,35,2,10,0,3,0.0,15.0,0.0,336907024
1,1955,1,46,1,2,0,1,0.0,0.0,0.0,336907024
2,1956,1,49,4,11,0,2,0.0,28.0,0.0,336907024
3,1957,1,49,4,14,0,6,0.0,26.0,0.0,336907024
4,1958,1,46,1,6,0,5,0.0,13.0,1.0,336907024


<h4>Manipulate Data</h4>

In [32]:
# Get data going back to latest year of interest
latestDataYear = 1980
targetBattingDF = battingDF[battingDF["yearID"] >=latestDataYear]
targetBattingDF.head()

,yearID,stint,G,R,H,HR,BB,IBB,SO,SB,fpID
87,1980,1,36,0,0,0,0,0.0,0.0,0.0,143883
88,1980,1,36,0,0,0,0,0.0,0.0,0.0,424800
89,1980,1,36,0,0,0,0,0.0,0.0,0.0,989035
90,1980,1,36,0,0,0,0,0.0,0.0,0.0,2809975
91,1980,1,36,0,0,0,0,0.0,0.0,0.0,5323248


In [33]:
# Combine the stats of players with multiple stints in a year
combinedStints = targetBattingDF.groupby(["yearID", "fpID"]).sum().reset_index()
combinedStints.head()

,yearID,fpID,stint,G,R,H,HR,BB,IBB,SO,SB
0,1980,1000,1,36,0,0,0,0,0.0,0.0,0.0
1,1980,1008,1,120,18,91,2,28,8.0,36.0,3.0
2,1980,1015,1,77,0,1,0,0,0.0,6.0,0.0
3,1980,1024,1,147,70,144,14,32,2.0,56.0,1.0
4,1980,1035,1,158,83,170,18,49,6.0,73.0,7.0


In [34]:
# Check to make sure combining Stints worked properly
totalRecords = targetBattingDF.shape[0]
combinedRecords = combinedStints.shape[0]

multipleStints = targetBattingDF.loc[targetBattingDF["stint"] > 1]
multipleStintRecords = multipleStints.shape[0]

if totalRecords - multipleStintRecords == combinedRecords:
    print("Success")
else:
    print("Failure")

Success


In [35]:
# Get Player Info

#just the player ID and fpID
fPlayersDF1 = fPlayersDF[["playerID", "fpID"]]
# Merge players with fplayers1
playerMergedDF = pd.merge(playersDF, fPlayersDF1, on = ["playerID"])
playerMergedDF.head()

,playerID,birthYear,nameFirst,nameLast,debut,finalGame,fpID
0,aardsda01,1981.0,David,Aardsma,2004-04-06,2015-08-23,266506624
1,aardsda01,1981.0,David,Aardsma,2004-04-06,2015-08-23,332151624
2,aardsda01,1981.0,David,Aardsma,2004-04-06,2015-08-23,362751115
3,aardsda01,1981.0,David,Aardsma,2004-04-06,2015-08-23,417304183
4,aardsda01,1981.0,David,Aardsma,2004-04-06,2015-08-23,451903563


In [36]:
# Create DF with  Years fields

playerYearsDF = playerMergedDF
playerYearsDF["debutYear"] = playerYearsDF["debut"].dt.year
playerYearsDF["finalYear"] = playerYearsDF["finalGame"].dt.year
playerYearsDF["totalYears"] = playerYearsDF.finalYear + 1 - playerYearsDF.debutYear
playerYearsDF = playerYearsDF[["fpID", "playerID", "birthYear", "debutYear", "finalYear", "totalYears"]]
playerYearsDF.head()

,fpID,playerID,birthYear,debutYear,finalYear,totalYears
0,266506624,aardsda01,1981.0,2004,2015,12
1,332151624,aardsda01,1981.0,2004,2015,12
2,362751115,aardsda01,1981.0,2004,2015,12
3,417304183,aardsda01,1981.0,2004,2015,12
4,451903563,aardsda01,1981.0,2004,2015,12


In [37]:
# Merge Player Years data with batting data

mergedBatting = pd.merge(combinedStints, playerYearsDF, on = ["fpID"])
mergedBatting.head()

,yearID,fpID,stint,G,R,H,HR,BB,IBB,SO,SB,playerID,birthYear,debutYear,finalYear,totalYears
0,1980,1000,1,36,0,0,0,0,0.0,0.0,0.0,barkele01,1955.0,1976,1987,12
1,1981,1000,1,22,0,0,0,0,0.0,0.0,0.0,barkele01,1955.0,1976,1987,12
2,1982,1000,1,33,0,0,0,0,0.0,0.0,0.0,barkele01,1955.0,1976,1987,12
3,1983,1000,3,30,1,1,0,0,0.0,5.0,0.0,barkele01,1955.0,1976,1987,12
4,1984,1000,1,21,2,2,0,6,0.0,19.0,0.0,barkele01,1955.0,1976,1987,12


In [38]:
# Normalize statistics to be per game

# Divide statistics by games played
mergedBatting["RpG"] = mergedBatting.R / mergedBatting.G
mergedBatting["HpG"] = mergedBatting.H / mergedBatting.G
mergedBatting["HRpG"] = mergedBatting.HR / mergedBatting.G
mergedBatting["BBpG"] = (mergedBatting.BB + mergedBatting.IBB) / mergedBatting.G
mergedBatting["SOpG"] = mergedBatting.SO / mergedBatting.G
mergedBatting["SBpG"] = mergedBatting.SB / mergedBatting.G

#Limit to only columns of interest
desiredCols = ["yearID", "fpID", "G","RpG", "HpG", "HRpG", "BBpG", "SOpG", "SBpG",
               "birthYear", "debutYear", "finalYear", "totalYears"]
mergedBatting = mergedBatting[desiredCols]
mergedBatting.head()

,yearID,fpID,G,RpG,HpG,HRpG,BBpG,SOpG,SBpG,birthYear,debutYear,finalYear,totalYears
0,1980,1000,36,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1955.0,1976,1987,12
1,1981,1000,22,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1955.0,1976,1987,12
2,1982,1000,33,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1955.0,1976,1987,12
3,1983,1000,30,0.033333,0.033333,0.0,0.000000,0.166667,0.0,1955.0,1976,1987,12
4,1984,1000,21,0.095238,0.095238,0.0,0.285714,0.904762,0.0,1955.0,1976,1987,12


In [39]:
# Add age and careerYears columns

mergedBatting["careerYear"] = mergedBatting.yearID + 1 - mergedBatting.debutYear
mergedBatting["age"] = mergedBatting.yearID - mergedBatting.birthYear

mergedBatting.head(5)

,yearID,fpID,G,RpG,HpG,HRpG,BBpG,SOpG,SBpG,birthYear,debutYear,finalYear,totalYears,careerYear,age
0,1980,1000,36,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1955.0,1976,1987,12,5,25.0
1,1981,1000,22,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1955.0,1976,1987,12,6,26.0
2,1982,1000,33,0.000000,0.000000,0.0,0.000000,0.000000,0.0,1955.0,1976,1987,12,7,27.0
3,1983,1000,30,0.033333,0.033333,0.0,0.000000,0.166667,0.0,1955.0,1976,1987,12,8,28.0
4,1984,1000,21,0.095238,0.095238,0.0,0.285714,0.904762,0.0,1955.0,1976,1987,12,9,29.0


<h4>Now, extract previous year data to be added to DataFrame</h4>

In [40]:
#testing ability to grab desired column value

currentYear = 1984
fpID = 1000

oneYearRow = mergedBatting.loc[(mergedBatting["yearID"] == currentYear - 1) & (mergedBatting["fpID"] == fpID)]
oneYearRpG = oneYearRow["RpG"].values[0]
print(oneYearRpG)


0.03333333333333333


In [64]:
# get a dataframe that will only hold valid inputs 
#  to get historical data for

# get only data from latestDataYear + 2
#  that way we can grab 2 years previous data
validHistoric = mergedBatting.loc[mergedBatting["yearID"] >= latestDataYear + 2]

# get only players who are in at least their 3rd career year and who played at least 4
#  we are using 3 years data to predict a 4th year value
validHistoric = validHistoric.loc[(validHistoric["careerYear"] >= 3) & (validHistoric["totalYears"] >= 4)]

# Get only data prior to 2019 season 
#  we can't supervise a model based of  2019 data becaus ewe don't have 2020 data to check it against
validHistoric = validHistoric.loc[validHistoric["yearID"] < 2019]
validHistoric.sort_values(by = ["yearID"], ascending = False).head()

# Remove players final years
#  we can't use 'next year' results to supervise when they don't play a next year
validHistoric2 = validHistoric.loc[validHistoric["yearID"] != validHistoric["finalYear"]]
validHistoric2.head(15)

,yearID,fpID,G,RpG,HpG,HRpG,BBpG,SOpG,SBpG,birthYear,debutYear,finalYear,totalYears,careerYear,age
2,1982,1000,33,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1955.0,1976,1987,12,7,27.0
3,1983,1000,30,0.033333,0.033333,0.000000,0.000000,0.166667,0.000000,1955.0,1976,1987,12,8,28.0
4,1984,1000,21,0.095238,0.095238,0.000000,0.285714,0.904762,0.000000,1955.0,1976,1987,12,9,29.0
5,1985,1000,20,0.000000,0.000000,0.000000,0.000000,0.350000,0.000000,1955.0,1976,1987,12,10,30.0
9,1982,1008,118,0.288136,0.805085,0.025424,0.389831,0.338983,0.033898,1955.0,1978,1989,12,5,27.0
10,1983,1008,134,0.320896,0.940299,0.014925,0.574627,0.179104,0.007463,1955.0,1978,1989,12,6,28.0
11,1984,1008,95,0.273684,0.705263,0.042105,0.389474,0.263158,0.010526,1955.0,1978,1989,12,7,29.0
12,1985,1008,70,0.171429,0.600000,0.000000,0.328571,0.171429,0.000000,1955.0,1978,1989,12,8,30.0
13,1986,1008,64,0.171875,0.562500,0.000000,0.250000,0.156250,0.015625,1955.0,1978,1989,12,9,31.0
14,1987,1008,37,0.108108,0.378378,0.027027,0.459459,0.405405,0.000000,1955.0,1978,1989,12,10,32.0


In [63]:
validHistoric.head(15)

,yearID,fpID,G,RpG,HpG,HRpG,BBpG,SOpG,SBpG,birthYear,debutYear,finalYear,totalYears,careerYear,age
2,1982,1000,33,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1955.0,1976,1987,12,7,27.0
3,1983,1000,30,0.033333,0.033333,0.000000,0.000000,0.166667,0.000000,1955.0,1976,1987,12,8,28.0
4,1984,1000,21,0.095238,0.095238,0.000000,0.285714,0.904762,0.000000,1955.0,1976,1987,12,9,29.0
5,1985,1000,20,0.000000,0.000000,0.000000,0.000000,0.350000,0.000000,1955.0,1976,1987,12,10,30.0
6,1987,1000,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1955.0,1976,1987,12,12,32.0
9,1982,1008,118,0.288136,0.805085,0.025424,0.389831,0.338983,0.033898,1955.0,1978,1989,12,5,27.0
10,1983,1008,134,0.320896,0.940299,0.014925,0.574627,0.179104,0.007463,1955.0,1978,1989,12,6,28.0
11,1984,1008,95,0.273684,0.705263,0.042105,0.389474,0.263158,0.010526,1955.0,1978,1989,12,7,29.0
12,1985,1008,70,0.171429,0.600000,0.000000,0.328571,0.171429,0.000000,1955.0,1978,1989,12,8,30.0
13,1986,1008,64,0.171875,0.562500,0.000000,0.250000,0.156250,0.015625,1955.0,1978,1989,12,9,31.0


In [59]:
# nums = [83,84, 85,86,87, 88, 89, 90, 91, 92, 95, 96, 97]
# lastYear = nums[-1]
# gap = 0
# gapTimes = 0
# for num in nums:
#     print(f"===== {num} =====")
#     if num >= lastYear:
#         continue
#     if gapTimes == 2:
#         print("Executing 2")
#         prev2 = num - 2 - gap
#         prev1 = num - 1 - gap
#         next1 = num + 1
#         gapTimes = 1
#     elif gapTimes == 1:
#         print("Executing 1")
#         prev2 = num - 2 - gap
#         prev1 = num - 1
#         next1 = num + 1
#         gapTimes = 0
#     else:
#         prev2 = num - 2
#         prev1 = num - 1
#         next1 = num + 1
    
#     while next1 not in nums and next1 <lastYear:
#         print(f"Missing {next1}")
#         next1 += 1
#         gapTimes = 2
#         gap = next1 - num - 1
    
#     print(prev2, prev1, num, next1)


In [ ]:
# # Practice looping though dataframes
# sampHistoric = validHistoric.head(10)

# # Set up columns for the new data to be entered into
# sampHistoric["prevTwo"] = ""
# sampHistoric["prevOne"] = ""
# sampHistoric["nextOne"] = ""
# stat = "HpG"
# print("=================")
# gap = 0
# gapTimes = 0
# old_player = 0
# for index, row in sampHistoric.iterrows():
#     player = row["fpID"]
#     if player != old_player:
#         print("NEW PLAYER")
#         gap = 0
#         gapTimes = 0
#         old_player = player
#     print(f"playerID: {player}")
#     yr = row["yearID"]
#     print(f"===== {yr} =====")
#     finalYear = row["finalYear"]
#     if yr >= finalYear:
#         continue
#     if gapTimes == 2:
#         print("Executing 2")
#         prevTwoYear = yr - 2 - gap
#         prevOneYear = yr - 1 - gap
#         nextYear = yr + 1
#         gapTimes = 1
#     elif gapTimes == 1:
#         print("Executing 1")
#         prevTwoYear = yr - 2 - gap
#         prevOneYear = yr - 1
#         nextYear = yr + 1
#         gapTimes = 0
#     else:
#         prevTwoYear = yr - 2
#         prevOneYear = yr - 1
#         nextYear = yr + 1

    

#     prevTwoRow = mergedBatting.loc[(mergedBatting["yearID"] == prevTwoYear)
#                                      & (mergedBatting["fpID"] == player)]
#     prevOneRow = mergedBatting.loc[(mergedBatting["yearID"] == prevOneYear)
#                                              & (mergedBatting["fpID"] == player)]
#     nextYearRow = mergedBatting.loc[(mergedBatting["yearID"] == nextYear)
#                                              & (mergedBatting["fpID"] == player)]
#     while nextYearRow.empty:
#         print(f"{nextYear} Missing")
#         nextYear += 1
#         nextYearRow = mergedBatting.loc[(mergedBatting["yearID"] == nextYear)
#                                              & (mergedBatting["fpID"] == player)]
#         gapTimes = 2
#         gap = nextYear - yr - 1
    
#     prevTwoStat = prevTwoRow[stat].values[0]
#     prevOneStat = prevOneRow[stat].values[0]    
#     nextYearStat = nextYearRow[stat].values[0]    
    
#     sampHistoric.at[index, "prevTwo"] = prevTwoStat
#     sampHistoric.at[index, "prevOne"] = prevOneStat
#     sampHistoric.at[index, "nextOne"] = nextYearStat
    
#     print(f"{prevTwoYear}: {prevTwoStat}")
#     print(f"{prevOneYear}: {prevOneStat}")
#     print(f"{yr}: {sampHistoric.loc[index, stat]}")
#     print(f"{nextYear}: {nextYearStat}")
          
#     print("=================")

        
# sampHistoric.head()


In [66]:
# run the above funtion on the entire 

# Designate Desired Stat
stat = "RpG"
# Set up 
validHistoricRpG = validHistoric2

# Set up columns for the new data to be entered into
validHistoricRpG["prevTwo"] = ""
validHistoricRpG["prevOne"] = ""
validHistoricRpG["nextOne"] = ""
stat = "RpG"
# print("=================")
gap = 0
gapTimes = 0
old_player = 0
rows_done = 0
for index, row in validHistoricRpG.iterrows():
    rows_done += 1
    
    player = row["fpID"]
    if player != old_player:
#         print("NEW PLAYER")
        gap = 0
        gapTimes = 0
        old_player = player
#     print(f"playerID: {player}")
    yr = row["yearID"]
#     print(f"===== {yr} =====")
    finalYear = row["finalYear"]
    if yr >= finalYear:
        continue
    if gapTimes == 2:
#         print("Executing 2")
        prevTwoYear = yr - 2 - gap
        prevOneYear = yr - 1 - gap
        nextYear = yr + 1
        gapTimes = 1
    elif gapTimes == 1:
#         print("Executing 1")
        prevTwoYear = yr - 2 - gap
        prevOneYear = yr - 1
        nextYear = yr + 1
        gapTimes = 0
    else:
        prevTwoYear = yr - 2
        prevOneYear = yr - 1
        nextYear = yr + 1

    

    prevTwoRow = mergedBatting.loc[(mergedBatting["yearID"] == prevTwoYear)
                                     & (mergedBatting["fpID"] == player)]
    prevOneRow = mergedBatting.loc[(mergedBatting["yearID"] == prevOneYear)
                                             & (mergedBatting["fpID"] == player)]
    nextYearRow = mergedBatting.loc[(mergedBatting["yearID"] == nextYear)
                                             & (mergedBatting["fpID"] == player)]
    while nextYearRow.empty:
#         print(f"{nextYear} Missing")
        nextYear += 1
        nextYearRow = mergedBatting.loc[(mergedBatting["yearID"] == nextYear)
                                             & (mergedBatting["fpID"] == player)]
        gapTimes = 2
        gap = nextYear - yr - 1
    
    prevTwoStat = prevTwoRow[stat].values[0]
    prevOneStat = prevOneRow[stat].values[0]    
    nextYearStat = nextYearRow[stat].values[0]    
    
    validHistoricRpG.at[index, "prevTwo"] = prevTwoStat
    validHistoricRpG.at[index, "prevOne"] = prevOneStat
    validHistoricRpG.at[index, "nextOne"] = nextYearStat
    
    print(rows_done)
    
#     print(f"{prevTwoYear}: {prevTwoStat}")
#     print(f"{prevOneYear}: {prevOneStat}")
#     print(f"{yr}: {validHistoricRpG.loc[index, stat]}")
#     print(f"{nextYear}: {nextYearStat}")
          
#     print("=================")

        
validHistoricRpG.head()

C:\Users\Vega\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\Vega\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\Vega\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231


IndexError: index 0 is out of bounds for axis 0 with size 0

In [67]:
mergedBatting.iloc[[230,235]].head()

,yearID,fpID,G,RpG,HpG,HRpG,BBpG,SOpG,SBpG,birthYear,debutYear,finalYear,totalYears,careerYear,age
230,1989,1783,30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1951.0,1975,1992,18,15,38.0
235,1981,1840,97,0.546392,1.072165,0.154639,0.268041,0.731959,0.020619,1952.0,1975,1985,11,7,29.0


In [68]:
validHistoricRpG.iloc[[230, 235]].head()

,yearID,fpID,G,RpG,HpG,HRpG,BBpG,SOpG,SBpG,birthYear,debutYear,finalYear,totalYears,careerYear,age,prevTwo,prevOne,nextOne
327,1982,2443,34,0.205882,0.205882,0.088235,0.176471,0.411765,0.000000,1957.0,1980,1987,8,3,25.0,,,
335,1982,2599,137,0.423358,0.773723,0.153285,0.525547,0.532847,0.043796,1954.0,1976,1988,13,7,28.0,,,
